In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from skimage.metrics import structural_similarity as ssim
from PIL import Image

SEED = 42
np.random.ssed(SEED)

In [ ]:
# Load dataset
def load_data(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            img_path = os.path.join(directory, filename)
            img = img.resize((28, 28))  # Resize to 28x28 if needed
            img = np.array(img) / 255.0  # Normalize
            images.append(img)
    images = np.expand_dims(images, axis=-1)  # Add channel dimension
    return np.array(images)

In [ ]:
# Define the generator model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=100),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(28 * 28 * 1, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])
    return model

In [ ]:
# Define the discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
# Compile models
def compile_models(generator, discriminator):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False

    gan_input = layers.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = tf.keras.models.Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')

    return gan

In [ ]:
# Train the GAN
def train_gan(generator, discriminator, gan, data, epochs, batch_size=128):
    epochs = 10
    for epoch in range(epochs):
        for _ in range(data.shape[0] // batch_size):
            # Train discriminator
            noise = np.random.normal(0, 1, (batch_size, 100))
            generated_images = generator.predict(noise)
            real_images = data[np.random.randint(0, data.shape[0], batch_size)]

            labels_real = np.ones((batch_size, 1))
            labels_fake = np.zeros((batch_size, 1))

            d_loss_real = discriminator.train_on_batch(real_images, labels_real)
            d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, 100))
            labels_gan = np.ones((batch_size, 1))

            g_loss = gan.train_on_batch(noise, labels_gan)

        print(f"Epoch {epoch + 1}/{epochs}, Discriminator Loss: {d_loss_real[0] + d_loss_fake[0]}, Generator Loss: {g_loss}")

In [ ]:
# Generate images and validate with SSIM
def validate_gan(generator, data, num_images=10):
    noise = np.random.normal(0, 1, (num_images, 100))
    generated_images = generator.predict(noise)
    real_images = data[np.random.randint(0, data.shape[0], num_images)]

    ssim_scores = [ssim(real_images[i].reshape(28, 28), generated_images[i].reshape(28, 28)) for i in range(num_images)]

    for i, score in enumerate(ssim_scores):
        print(f"SSIM for image {i+1}: {score}")

In [ ]:
# Main function
def main():
    data = load_data('path_to_fitz_directory')
    generator = build_generator()
    discriminator = build_discriminator()
    gan = compile_models(generator, discriminator)
    train_gan(generator, discriminator, gan, data, epochs=10)
    validate_gan(generator, data)

if __name__ == "__main__":
    main()